<a href="https://colab.research.google.com/github/kenclements/praxis/blob/main/adaboost-twos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!nvidia-smi

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb

In [4]:
file = pd.read_csv('/content/drive/MyDrive/Praxis/data/TWOS/email_ano/User1b.csv')

In [5]:
file

,label,body_anon,liwc_WC,liwc_Analytic,liwc_Clout,liwc_Authentic,liwc_Tone,liwc_WPS,liwc_Sixltr,liwc_Dic,...,liwc_Comma,liwc_Colon,liwc_SemiC,liwc_QMark,liwc_Exclam,liwc_Dash,liwc_Quote,liwc_Apostro,liwc_Parenth,liwc_OtherP
0,NORMAL,Hey,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NORMAL,Hey guys,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NORMAL,Yeah I think we should: 71J871LWL3) Backup UAH...,42.0,22.77,82.96,6.50,3.31,14.00,14.29,78.57,...,0.0,2.38,0.0,2.38,0.0,0.0,0.0,0.0,7.14,0.0
3,NORMAL,Okay so as protection we will just clear all i...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NORMAL,Can M5P10WM4K7 the whole desktop with excel fi...,38.0,12.16,96.70,50.35,97.58,9.50,7.89,81.58,...,0.0,0.00,0.0,2.63,0.0,0.0,0.0,0.0,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,ANOMALY,I think it is a good idea if you guys can orga...,69.0,46.46,87.68,27.39,78.80,8.62,15.94,91.30,...,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0
156,ANOMALY,Hey guys,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
157,ANOMALY,Tue =WG4V688FUG V1AGEWA8CT Wed =WG4V688FUG OAP...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
158,ANOMALY,Hey friends,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
import nltk
from nltk.corpus import stopwords

stops = set(stopwords.words('english'))
print(stops)

LookupError: ignored